In [1]:
individus = 100 # nombre d'individus dans le graphe

# taille des clusters du graphe (min/max)
foyer = (2, 5)
communaute = (4, 5) # nb de foyers dans une communauté

# probas initialisation
rencontreFoyer = 0.9 # proba de rencontrer un autre membre du foyer
rencontreCommunaute = 0.01 # proba de rencontre un autre membre de la communauté
rencontreNational = 0.001

# probas


In [3]:
# Initialisation du graphe

foyers = []
communautes = []

adj = [[] for l in range(individus)]
adj[0].append({"noeud": 3, "proba": 0.2})

In [4]:
# Passage au jour suivant du graphe
def step(graphe):
    pass

In [5]:
# Affichage
